In [181]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [182]:
#dir_path = "../process/"
#file= "external_test_data"# "train_data" #
fold_N=0
if fold_N!=None:
    dir_path="../process/folds_process/"
file= f"train_fold_{fold_N}" #f"val_fold_{fold_N}" #
logs_df = pd.read_csv(f"{dir_path}3_21{file}_heuristic_tagging.csv", encoding='utf-8')
print(set(logs_df["source"]))
print(logs_df.shape)
logs_df.head()

{'Hadoop', 'Apache', 'BGL', 'Zookeeper', 'Android'}
(2160, 13)


,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:18.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,-1
1,jk2_init() Found child 6734 in scoreboard slot 9,Informativo,2005-12-04 04:51:38.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Monitoreo,Único,48,4.0,-1
2,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 04:51:52.000,6,Apache,workerenv init ok httpd conf properties,Depuración,Aplicación,Monitoreo,Recurrente esperado,55,4.0,-1
3,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:55.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,-1
4,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 04:52:12.000,6,Apache,workerenv init ok httpd conf properties,Depuración,Aplicación,Monitoreo,Recurrente esperado,55,4.0,-1


In [183]:
# Función para calcular la prioridad de un evento
def calcular_prioridad(row):
    prioridad = 0

    # Ponderación por Severidad
    if row['Severidad'] == 'Crítico':
        prioridad += 3  # Alta prioridad
    elif row['Severidad'] == 'Advertencia':
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1  # Baja prioridad

    # Ponderación por Origen
    if row['Origen'] in ['Sistema operativo', 'Aplicación', ]:
        prioridad += 3  # Alta prioridad para estos orígenes
    elif row['Origen'] in ['Gestión de energía', 'Conectividad']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1 

    # Ponderación por Tipo de Evento
    if row['TipoEvento'] in ['Errores', 'Parada de procesos']:
        prioridad += 3  # Alta prioridad
    elif row['TipoEvento'] in  ['Eventos']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1 
        
    # Ponderación por Frecuencia
    if row['Frecuencia'] in ['Recurrente anómalo']:
        prioridad += 3  # Alta prioridad
    elif row['Frecuencia'] in ['Único']:
        prioridad += 2  # Prioridad media
    else:
        prioridad += 1  # Baja prioridad

    # Ponderación por Anomaly
    if row['Anomaly'] == -1:
        prioridad += 2  # Alta prioridad
    else:
        prioridad += 1  # Baja prioridad
        
    return prioridad

# Aplicar la función para calcular la prioridad
logs_df['Prioridad'] = logs_df.apply(calcular_prioridad, axis=1)


In [184]:
T=logs_df[logs_df["Level"]=="Error"].copy()
print("Error", np.min(T['Prioridad']), np.mean(T['Prioridad']),  np.max(T['Prioridad']))

T=logs_df[logs_df["Level"]=="Advertencia"].copy()
print("Advertencia", np.min(T['Prioridad']), np.mean(T['Prioridad']),  np.max(T['Prioridad']))

T=logs_df[logs_df["Level"]=="Informativo"].copy()
print("Informativo", np.min(T['Prioridad']), np.mean(T['Prioridad']),  np.max(T['Prioridad']))

Error 5 9.080555555555556 13
Advertencia 5 9.25 11
Informativo 5 6.951388888888889 9


In [185]:
# Asignar etiquetas de prioridad en función del puntaje
def asignar_prioridad_etiqueta(row):
    if row['Prioridad'] > 10:
        return 'Error'
    elif row['Prioridad'] > 7:
        return 'Advertencia'
    else:
        return 'Informativo'


logs_df['PrioridadEtiqueta'] = logs_df.apply(asignar_prioridad_etiqueta, axis=1)
logs_df.head()

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly,Prioridad,PrioridadEtiqueta
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:18.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,-1,12,Error
1,jk2_init() Found child 6734 in scoreboard slot 9,Informativo,2005-12-04 04:51:38.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Monitoreo,Único,48,4.0,-1,8,Advertencia
2,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 04:51:52.000,6,Apache,workerenv init ok httpd conf properties,Depuración,Aplicación,Monitoreo,Recurrente esperado,55,4.0,-1,8,Advertencia
3,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:51:55.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,-1,12,Error
4,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 04:52:12.000,6,Apache,workerenv init ok httpd conf properties,Depuración,Aplicación,Monitoreo,Recurrente esperado,55,4.0,-1,8,Advertencia


In [186]:
logs_df.to_csv(f"{dir_path}3_22{file}_heuristic_prediction.csv", index=False, encoding='utf-8')

In [187]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [188]:
print({file})
# Asumiendo que logs_df ya está definido y contiene las columnas 'Level' y 'PrioridadEtiqueta'
# Asegúrate de que ambos sean de tipo categórico (opcional pero recomendado)
logs_df['Level'] = logs_df['Level'].astype('category')
logs_df['PrioridadEtiqueta'] = logs_df['PrioridadEtiqueta'].astype('category')

# Extraer columnas
y_true = logs_df['Level']
y_pred = logs_df['PrioridadEtiqueta']

# Calcular accuracy
accuracy = accuracy_score(y_true, y_pred)

# Obtener etiquetas únicas combinadas para garantizar alineación
all_labels = sorted(set(y_true.unique()) | set(y_pred.unique()))

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred, labels=all_labels)

# Reporte de clasificación
report = classification_report(y_true, y_pred, labels=all_labels, output_dict=True)

# Convertir a DataFrame
report_df = pd.DataFrame(report).transpose()

# Mostrar resultados
print(f"Accuracy: {accuracy:.2f}\n")
print("Classification Report:")
print(report_df)
print("\nConfusion Matrix:")
print(pd.DataFrame(conf_matrix, index=all_labels, columns=all_labels))

{'train_fold_0'}
Accuracy: 0.66

Classification Report:
              precision    recall  f1-score     support
Advertencia    0.576471  0.816667  0.675862   720.00000
Error          0.792056  0.470833  0.590592   720.00000
Informativo    0.693820  0.686111  0.689944   720.00000
accuracy       0.657870  0.657870  0.657870     0.65787
macro avg      0.687449  0.657870  0.652133  2160.00000
weighted avg   0.687449  0.657870  0.652133  2160.00000

Confusion Matrix:
             Advertencia  Error  Informativo
Advertencia          588     89           43
Error                206    339          175
Informativo          226      0          494


In [189]:
dir_path_model = "../model/"
pd.DataFrame(report).transpose().to_csv(f"{dir_path_model}/rh_report_fold_{fold_N}.csv")
